<a href="https://colab.research.google.com/github/workmatechiho-source/blank-app/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ---------------------------------
# PYTHON IMPORTS
# ---------------------------------

import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import FixedLocator, FuncFormatter, NullLocator, NullFormatter

In [3]:
# ---------------------------------
# INPUTS
# ---------------------------------

p = dict(
    # Rock / load
    gamma_kN_m3 = 24.0,        # rock unit weight [kN/m^3]
    load_model  = "overbreak", # "wedge" or "overbreak"
    overbreak_m = 0.2,        # loosened rock thickness h [m] for overbreak
    q_kPa  = 0.0,              # surcharge load [kPa]
    qw_kPa = 0.0,              # groundwater pressure [kPa]
    LF = 1.5,                  # load factor

    # Geometry
    s0_m = 1.0,                # reference bolt spacing [m]
    t_m  = 0.10,               # shotcrete thickness [m]
    c_m  = 0.15,               # plate width [m]
    CL_m = 0.0,               # corrosion / cover loss [m]

    # Adhesion
    sigma_a_poor_MPa = 0.1,   # adhesion stress [MPa] - poor case
    a_poor_m = 0.05,           # adhesion length [m] - poor case
    sigma_a_good_MPa = 0.1,   # adhesion stress [MPa] - good case
    a_good_m = 0.05,           # adhesion length [m] - good case

    # Material
    f_c_MPa = 40.0,            # compressive strength [MPa]
    f_eq_MPa = 2.0,            # equivalent residual flexural [MPa] (beam tests)
    k_f = 1.0,                 # fibre efficiency
    SRF = 0.6,                 # strength reduction factor
    rho_sc_kN_m3 = 22.5,       # density shotcrete [kN/m^3]

    # Punching
    # will compute sigma_ps via Eq. (17)+(18) rather than fixed cap
    # Plot sweep
    s_min = 0.5, s_max = 4.0, n_pts = 60
)

In [4]:
# ---------------------------------
# DERIVED MATERIAL PROPERTIES (2017 paper)
# ---------------------------------

# Flexural strength (Eq. (10))
sigma_fl_c_MPa = 0.6 * math.sqrt(p["f_c_MPa"])        # plain flexural
sigma_fl_MPa   = min(sigma_fl_c_MPa, p["f_eq_MPa"])   # capped by beam test

# Effective flexural strength for bending capacity (B&M strip usage)
sigma_fl_eff_MPa = 0.5 * sigma_fl_MPa                 # capacity uses 0.5·σ_fl

# Effective thickness with corrosion allowance (t_cor = CL_m)
t_eff_m = max(p["t_m"] - p.get("CL_m", 0.0), 0.0)

# Direct shear (Eq. (12)+(13))
tau_c_MPa = 0.15 * (p["f_c_MPa"] ** (1/3))
k_d = max(1.6 - t_eff_m, 1.0)                         # <-- use t_eff, not t_m
tau_f_MPa = (0.12 * p["k_f"] * k_d * p["f_eq_MPa"]) / 0.37
tau_ds_MPa = tau_c_MPa + tau_f_MPa

# Diagonal tensile strength (Eq. (11))
sigma_dd_MPa = 0.4 * math.sqrt(p["f_c_MPa"])

# Punching stress (Eq. (17)+(18))
sigma_ps_c_MPa = 0.17 * math.sqrt(p["f_c_MPa"])
sigma_ps_f_MPa = 0.37 * p["k_f"] * (sigma_fl_MPa / (t_eff_m if t_eff_m > 0 else 1.0))
sigma_ps_MPa   = sigma_ps_c_MPa + sigma_ps_f_MPa

In [5]:
# ---------------------------------
# HELPER FUNCTIONS
# ---------------------------------

def rock_load_components_kN(s, P):
    """Return rock, self-weight, surcharge, and groundwater components (all kN)."""
    W_rock = P["gamma_kN_m3"] * P["overbreak_m"] * s**2
    W_sc   = P["rho_sc_kN_m3"] * P["t_m"] * s**2
    W_q    = P.get("q_kPa", 0.0)  * s**2
    W_qw   = P.get("qw_kPa", 0.0) * s**2
    return W_rock, W_sc, W_q, W_qw

def rock_load_kN(s, P):
    """Panel load W (Eq. (1) wedge or overbreak block)."""
    if P["load_model"].lower() == "overbreak":
        W_rock, W_sc, W_q, W_qw = rock_load_components_kN(s, P)
        return W_rock + W_sc + W_q + W_qw
    return P["gamma_kN_m3"] * math.sqrt(6.0) * s**3 / 6.0  # wedge (Eq. (1))

def adhesion_capacity_kN(s, sigma_a_MPa, a_m):
    """Adhesion (Eq. (2))."""
    return 4.0 * sigma_a_MPa * 1000.0 * s * a_m

def direct_shear_capacity_kN(s, P, tau_ds_MPa):
    """Direct shear capacity (Eq. 12+13, 2017 paper).
    Uses reduced thickness (t_eff = t - CL)."""
    t_eff = max(P["t_m"] - P.get("CL_m", 0.0), 0.0)
    return 4.0 * tau_ds_MPa * 1000.0 * s * t_eff



def flexural_capacity_kNm(s, P, sigma_fl_eff):
    t_eff = max(P["t_m"] - P.get("CL_m", 0.0), 0.0)  # deduct CL only in capacity
    return sigma_fl_eff * 1000.0 * (t_eff**2) / 6.0 * (s / 2.0)


def flexural_demand_kNm(s, P):
    # Mathcad convention: bending uses rock + q only, then 0.65 connection factor
    W_rock, W_sc, W_q, W_qw = rock_load_components_kN(s, P)
    W_flex = W_rock + W_q
    m_conn = 0.65 if P.get("mc_connected", True) else 1.0
    shape  = ((s - 0.7 * P["c_m"]) / s) ** 2
    return P["LF"] * W_flex * s / 8.0 * shape * m_conn

# --- Punching ---
def V_shear_kN(W_kN, s_m, c_m):
    """Critical shear (Eq. (5))."""
    return W_kN * (1.0 - (c_m / s_m)**2)

def d_from_equilibrium_m(W_kN, s_m, c_m, sigma_dd_MPa):
    """Effective perimeter extension (Eq. (6))."""
    V = V_shear_kN(W_kN, s_m, c_m)
    return math.sqrt((c_m/4.0)**2 + (V/1000.0)/(4.0*sigma_dd_MPa)) - c_m/4.0


def punching_capacity_equilibrium_kN(P, W_kN_total, s_m):
    """
    Punching capacity per Christine et al. (2017):
      - d from equilibrium with sigma_ds = 0.4*sqrt(fc) [MPa]
      - geometry uses c/2 inside the root
      - V uses mechanical loads only (rock + self-weight + surcharge), excludes groundwater
      - capacity uses perimeter 4(c + t + d)*t and sigma_ps ≈ 2.15 MPa (calibrated)
    """
    c = P["c_m"]; t = P["t_m"]

    # --- 1) diagonal tensile strength for equilibrium
    sigma_ds_MPa = 0.4 * math.sqrt(P["f_c_MPa"])   # ≈ 2.53 MPa for fc=40

    # --- 2) build V from components, excluding groundwater for punching equilibrium
    W_rock, W_sc, W_q, W_qw = rock_load_components_kN(s_m, P)
    V_kN = (W_rock + W_sc + W_q) * (1.0 - (c / s_m)**2)

    # --- 3) d from equilibrium (use c/2)
    c2 = c / 2.0
    d_m = math.sqrt(c2**2 + (V_kN/1000.0)/(4.0 * sigma_ds_MPa)) - c2

    # --- 4) punching stress (calibrated constant)
    sigma_ps = P.get("sigma_ps_cap_MPa", 2.15)  # MPa

    # --- 5) capacity with perimeter 4(c + t + d)*t
    Cps = sigma_ps * 1000.0 * 4.0 * (c + t + d_m) * t
    return Cps, d_m


def fos(cap, dem):
    return cap/dem if dem > 0 else float("inf")

In [6]:
# ---------------------------------
# SUMMARY TABLE at s0
# ---------------------------------

s0  = p["s0_m"]
LF  = p["LF"]

W0  = rock_load_kN(s0, p)
W0f = LF * W0

Wrock, Wsc, Wq, Wqw = rock_load_components_kN(s0, p)
Wflex = Wrock + Wq

Ca_poor = adhesion_capacity_kN(s0, p["sigma_a_poor_MPa"], p["a_poor_m"])
Ca_good = adhesion_capacity_kN(s0, p["sigma_a_good_MPa"], p["a_good_m"])
Cds     = direct_shear_capacity_kN(s0, p, tau_ds_MPa)
Cfl = flexural_capacity_kNm(s0, p, sigma_fl_eff_MPa)

Md      = flexural_demand_kNm(s0, p)
Mo      = Md / LF
Cps, d_m = punching_capacity_equilibrium_kN(p, W0, s0)

SRF = p["SRF"]

summary = pd.DataFrame([
    ["Adhesion (poor)",   Ca_poor, SRF*Ca_poor, W0,  W0f, fos(SRF*Ca_poor, W0f)],
    ["Adhesion (good)",   Ca_good, SRF*Ca_good, W0,  W0f, fos(SRF*Ca_good, W0f)],
    ["Direct Shear",      Cds,     SRF*Cds,     W0,  W0f, fos(SRF*Cds,     W0f)],
    ["Flexure",           Cfl,     SRF*Cfl,     Mo,  Md,  fos(SRF*Cfl,     Md )],
    ["Punching",          Cps,     SRF*Cps,     W0,  W0f, fos(SRF*Cps,     W0f)],
], columns=[
    "Check",
    "Capacity (kN or kN·m) - Ultimate",
    "Capacity (kN or kN·m) - Factored",
    "Demand (kN or kN·m) - Working",
    "Demand (kN or kN·m) - Factored",
    "FoS [-]"
])

display(
    summary.style.format({
        "Capacity (kN or kN·m) - Ultimate": "{:.2f}",
        "Capacity (kN or kN·m) - Factored": "{:.2f}",
        "Demand (kN or kN·m) - Working": "{:.2f}",
        "Demand (kN or kN·m) - Factored": "{:.2f}",
        "FoS [-]": "{:.2f}",
    })
)

,Check,Capacity (kN or kN·m) - Ultimate,Capacity (kN or kN·m) - Factored,Demand (kN or kN·m) - Working,Demand (kN or kN·m) - Factored,FoS [-]
0,Adhesion (poor),20.00,12.00,7.05,10.58,1.13
1,Adhesion (good),20.00,12.00,7.05,10.58,1.13
2,Direct Shear,594.39,356.63,7.05,10.58,33.72
3,Flexure,0.83,0.50,0.31,0.47,1.07
4,Punching,218.79,131.28,7.05,10.58,12.41
